In [214]:
import numpy as np
try:
    from IPython.core.display import clear_output
    have_ipython = True
except ImportError:
    have_ipython = False
import sys
import datetime
%matplotlib inline
import matplotlib.pyplot as plt

def plotCurve(train_mean, train_std,test_mean,test_std,sizes):
    plt.plot(sizes, train_mean, 
            color='blue', marker='o', 
            markersize=5, 
            label='training accuracy')
    plt.fill_between(sizes, 
                  train_mean + train_std,
                   train_mean - train_std, alpha=0.15, color='blue')

    plt.plot(sizes, test_mean, 
              color='green', linestyle='--', 
              marker='s', markersize=5, 
             label='validation accuracy')
    plt.fill_between(sizes, 
                      test_mean + test_std,
                     test_mean - test_std, 
                    alpha=0.15, color='green')
    plt.xlabel('x_range')
    plt.ylabel('Accuracy')
    plt.show()
def likehoodScore(proba,y):
    return np.sum(proba * y)/proba.shape[0]

def firstNScore(n, pred, y):
    backup = np.array(pred, copy =True)
    for r in range(pred.shape[0]):
        row = backup[r]
        s = np.sort(row)
        for c in range(pred.shape[1]):
            temp = backup[r][c]
            backup[r][c] = False
            for j in range(1,n+1):
                if temp == s[-j]:
                    backup[r][c] = True
                    break
    res = np.sum(np.logical_and(backup,y))/pred.shape[0]
    return res               

def oneHotDecode(self, X_sample):
    result=None
    fiPos = 0
    colIndex = 0
    while colIndex < X_sample.shape[1]:
        if fiPos < len(self.ohe.categorical_features) and colIndex == self.ohe.feature_indices_[fiPos]:                
            start = self.ohe.feature_indices_[fiPos]
            end_ = start+ self.ohe.n_values_[fiPos]
            #print("start{} end{}".format(start,end_))
            classes = np.argmax(X_sample[:,start:end_],axis=1).reshape(X_sample.shape[0],1)
            if result is None:
                result = classes
            else:
                result=np.hstack([result,classes])
            colIndex = end_
            fiPos = fiPos +1
        else:
            if result is None:
                result = X_sample[:,colIndex:colIndex+1]
            else:
                result=np.hstack([result, X_sample[:,colIndex:colIndex+1]])
            colIndex = colIndex +1
        
    return result 
def convertToDate(dayStamps):
    res = [] 
    for v in dayStamps:
        res.append(datetime.datetime.fromtimestamp(v*24*60*60))
    return res

#TODO confusion matrix ,e.g. 100 - 90 % precision
def _precisionClassify(df,proba, wins, c =0 ):
    for indx, v in enumerate(proba):
        row = 0
        col = 0
        if wins[indx] == c:
            col = 0
        else:
            col =1
        if v <0.2:
            row =6 
        elif v < 0.3 and  v >=0.2:
            row =5 
        elif v < 0.4 and v >= 0.3:
            row = 4 
        elif v < 0.5 and v >= 0.4:
            row = 3 
        elif v < 0.6 and v >= 0.5:
            row = 2 
        elif v < 0.8 and v >= 0.6:
            row = 1
        df.iloc[row,col] = df.iloc[row,col]+1 
    df[df.columns[2]] = df[df.columns[0]] /(df[df.columns[1]] + df[df.columns[0]])
    return df
    

def precisionMatrix(proba, y):
    rowHeader = ['>80','60-80','50-60','40-50','30-40','20-30','<20']
    df = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['h_Correct', 'h_Wrong','h_Precent'])
    hproba = proba[:,0]
    wins = np.argmax(y,axis=1)
    df = _precisionClassify(df,hproba,wins)
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['d_Correct', 'd_Wrong','d_Precent'])
    dproba = proba[:,1]
    df = df.join(_precisionClassify(temp,dproba,wins,c=1))
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['a_Correct', 'a_Wrong','a_Precent'])
    aproba = proba[:,2]
    df = df.join(_precisionClassify(temp,aproba,wins,c=2))
    return df
       
from datetime import date, timedelta
def futureTest(mlp, X,y,numOfWeek = 10,verbose=False):
    decoded = oneHotDecode(c, X)
    dates = convertToDate(decoded[:,c.dateColumn])
    dates = [d - timedelta(days=2) for d in dates]    
    weeks  = [ v.isocalendar()[1] for v in dates]

    thisWeek = weeks[-1]
    start = -1
    last = X.shape[0]
    index = -1
    w = 0
    sum_proba =None 
    sum_y =None
    sum_train_proba=None
    sum_train_y=None
    while w < numOfWeek:
        if thisWeek != weeks[index]:
            start = X.shape[0] +index+1
            X_train = X[0:start, :]
            X_test = X[start:last,:]
            y_train = y[0:start,:]
            y_test = y[start:last,:]
            mlp.fit(X_train,y_train)
            decoded = oneHotDecode(c,X_test)
            home = np.array([c.inverseTeamMapping(decoded[:,0])]).reshape(X_test.shape[0],1)
            away = np.array([c.inverseTeamMapping(decoded[:,1])]).reshape(X_test.shape[0],1)
            stack = np.hstack([home,away])
            proba = mlp.predict_proba(X_test)
            train_proba =mlp.predict_proba(X_train)
            errorIndx = np.argmax(proba,axis=1) != np.argmax(y_test,axis=1)
            if sum_proba is None:
                sum_proba = proba
                sum_y = y_test
                sum_train_proba = train_proba
                sum_train_y = y_train
            else:
                sum_proba = np.vstack([sum_proba,proba])
                sum_y = np.vstack([sum_y,y_test])
                sum_train_proba = np.vstack([sum_train_proba, train_proba])
                sum_train_y= np.vstack([sum_train_y, y_train])
            if verbose == True:
                print("week{}".format(w))
                print("numOftest {} , score {}".format(X_test.shape[0],mlp.score(X_test,y_test)))
                print(np.hstack([stack[errorIndx],proba[errorIndx],y_test[errorIndx]]))
                print("first2 : {}",firstNScore(2,proba,y_test))
            last = start
            thisWeek = weeks[index]
            w = w+1
        index = index -1
        
    print("summary")
    print("score:")
    score = firstNScore(1,sum_proba,sum_y)
    print(score)
    print("2like")
    print(firstNScore(2,sum_proba,sum_y))
    print(precisionMatrix(sum_proba,sum_y))
    
    return firstNScore(1,sum_train_proba,sum_train_y), score, 0


In [240]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
class FootballDataHelper:
    def __init__ (self, recentNum=5):
        self.win_mapping = {'H':0, 'D':1,'A':2}
        self.recentNum = recentNum
        self.df = None
        #self.hiddensCount = 2
       
        
    def readFootBallData(self,filename): 
        df = pd.read_csv(filename)
        df = df.drop(df.columns[range(23,df.shape[1])], axis=1)
        df = df.drop("Div",axis=1)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
        df['HTR'] = df['HTR'].map(self.win_mapping)
        df['FTR'] = df['FTR'].map(self.win_mapping)
        df= df.drop('Referee', 1)
        print(df.shape)
          #self.team = df['HomeTeam'].drop_duplicates()
        if self.df is None:
            self.df = df
        else:
            self.df = pd.concat([self.df,df])
            
        teams = self.df['HomeTeam'].drop_duplicates()
        teamMap = {}
        for index , v in enumerate(teams):
            teamMap[v] = index
        self.teamsMap = teamMap
 
    def getTeam(self,dataFrame, teamName):       
        return dataFrame[(dataFrame["HomeTeam"] == teamName) | (dataFrame["AwayTeam"] == teamName)]
 
        
    def previousRecords(self,team, date , recentNum):
        prev = team[( team["Date"] < date)]
        
        if prev.shape[0] < recentNum :
            #print("less than min Num")
            return None
        else:
            return prev.iloc[-recentNum:]
    def readPredict (self, filename):
        df = pd.read_csv(filename)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
        df['FTR'] = df['FTR'].map(self.win_mapping)
        return df
    
    
  
    def inverseTeamMapping (self, col):
        inverseMap ={}
        for name in self.teamsMap.keys():        
            inverseMap[self.teamsMap[name]] = name
        res =[]
        for idex, v in enumerate(col):
            res.append(inverseMap[v])
        return res

    def getH1(self):
        #recent matches (only win/loss/draw)
       
        
       # print (self.df)      
        X  = self.df.sort_values(by="Date")
        y = []
        for v in X['FTR']:
            y.append(range(3)==v)
       # print(y)
        resy=[]
        resx=[]
        print("h1:start format")
        for i in range(X.shape[0]):
            
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            x = X.iloc[i]
            homeName = x['HomeTeam']
            awayName = x['AwayTeam']
            homeTeam = self.getTeam(X,homeName)
            awayTeam = self.getTeam(X,awayName)
            prevHome = self.previousRecords(homeTeam,x['Date'])
            prevAway = self.previousRecords(awayTeam,x['Date'])
            if prevHome is None or prevAway is None:
               # print("{} skip".format(i))
                continue
            prevHomeWin = []
            prevAwayWin = []
            for v in prevHome['FTR']:
                prevHomeWin.append(range(3) == v)
            #print(prevHomeWin)
            for v in prevAway['FTR']:
                prevAwayWin.append(range(3) == v)
            pHHT=(prevHome['HomeTeam']==homeName).values
            pAHT=(prevAway['HomeTeam']==awayName).values
            tempX=[]
            for j in range(pHHT.shape[0]):
                tempX.append(np.append(prevHomeWin[j],pHHT[j]))
                
            for j in range(pAHT.shape[0]):
                tempX.append(np.append(prevAwayWin[j],pAHT[j]))
            resx.append(np.ravel(tempX))
            
            resy.append(y[i])
        print("finish")
        sys.stdout.flush()
        return np.array(resx), np.array(resy)
    def getH2(self):         
       #team based      
        X  = self.df.sort_values(by="Date")
        y = []
        for v in X['FTR']:
            y.append(range(3)==v)
       # print(y)
        resy=[]
        resx=[]
        print("h1:start format")
        X['HomeTeam'] = X['HomeTeam'].map(self.teamsMap)
        X['AwayTeam'] = X['AwayTeam'].map(self.teamsMap)
        nativeX = X[['HomeTeam','AwayTeam']].values
        #print(X)
        ohe = OneHotEncoder(categorical_features=[0,1])
        res = ohe.fit_transform(nativeX).toarray()
        #print(res)
        return res, np.array(y)
    def _getH3RecentMatches(self,x, X,teamName,recentNum):
        team = self.getTeam(X,teamName)
        prev = self.previousRecords(team,x['Date'],recentNum)
        if prev is None:
               return None
        prevHt=  prev['HomeTeam'].values
        prevAt=  prev['AwayTeam'].values   
        prevIsHome = []
        prevOther = []
        for i in range(recentNum):
            if prevHt[i] == teamName:
                prevIsHome.append(1)
                prevOther.append(prevAt[i])
            else:
                prevIsHome.append(0)
                prevOther.append(prevHt[i])
        wins = prev['FTR'].values       
        temp = np.array([prevIsHome,prevOther,wins]).T

        return np.ravel(temp)
        
    def getH3(self, recentNum):
        X  = self.df.sort_values(by="Date")
        X['HomeTeam'] = X['HomeTeam'].map(self.teamsMap)
        X['AwayTeam'] = X['AwayTeam'].map(self.teamsMap)
        y = []
        for v in X['FTR']:
            y.append(range(3)==v)
       # print(y)
        resy=[]
        resx=[]
        print("h3:start format")
        recents = []
        for i in range(X.shape[0]):
            
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            x = X.iloc[i]
            homeName = x['HomeTeam']
            awayName = x['AwayTeam']
            homeRecent = self._getH3RecentMatches(x,X,homeName, recentNum)
            awayRecent =self._getH3RecentMatches(x,X,awayName, recentNum)
            if homeRecent is None or awayRecent is None:
                   continue 
        #    print(homeName)
         #   print(homeRecent)
        #    print(awayName)
         #   print(awayRecent)
         #   return
            recents.append(np.hstack([homeName,awayName, homeRecent,awayRecent]))        
            resy.append(y[i])
        cols =np.hstack([[0,1],list(range(3,len(recents[0]),3))])
        print(cols)
        ohe = OneHotEncoder(categorical_features=cols)
        res = ohe.fit_transform(recents).toarray()
        print("finish")
        sys.stdout.flush()
        return res, np.array(resy)
   

    def getH4(self, recentNum, target=None):
        
        X  = self.df.sort_values(by="Date")
        isInput = False
        if target is None:
            target =X      
        else:
            if self.ohe is None:
                raise Exception("Not yet get train data")
            isInput = True
            target['HomeTeam']=target['HomeTeam'].map(self.teamsMap)
            target['AwayTeam']=target['AwayTeam'].map(self.teamsMap)
            pass
            
        X['HomeTeam'] = X['HomeTeam'].map(self.teamsMap)
        X['AwayTeam'] = X['AwayTeam'].map(self.teamsMap)
        target_date = (pd.to_numeric(target['Date'])/1e9/24/60/60).values
        y = []
        for v in target['FTR']:
            y.append(range(3)==v)
       # print(y)
        resy=[]
        resx=[]
        print("h4:start format")
        recents = []
        for i in range(target.shape[0]):
            
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            x = target.iloc[i]
            homeName = x['HomeTeam']
            awayName = x['AwayTeam']
            homeRecent = self._getH3RecentMatches(x,X,homeName, recentNum)
            awayRecent =self._getH3RecentMatches(x,X,awayName, recentNum)
            if homeRecent is None or awayRecent is None:
                   continue 
           # print(homeName)
           # print(homeRecent)
          #  print(awayName)
            #print(awayRecent)
            if recentNum == 0:
                recents.append(np.hstack([homeName,awayName,target_date[i]]))
            else:
                recents.append(np.hstack([homeName,awayName,target_date[i], homeRecent,awayRecent]))            
            print(len(recents[0]))
            return
            resy.append(y[i])
#        print(recents)
        if isInput==False:
            cols = None
            if recentNum == 0 :
                cols = np.array([0,1])
            else:
                cols =np.hstack([[0,1],list(range(4,len(recents[0]),3))])
            self.ohe = OneHotEncoder(categorical_features=cols)
            self.ohe.fit(recents)
            self.dateColumn = len(cols)
        res = self.ohe.transform(recents).toarray()
        print("finish")
        sys.stdout.flush()
        return res, np.array(resy)

In [241]:
c = FootballDataHelper(recentNum=4)
c.readFootBallData("E0_1314.csv")
c.readFootBallData("E0_1415.csv")
c.readFootBallData("E0 (1).csv")
print(c.teamsMap)

(380, 21)
(380, 21)
(293, 21)
{'QPR': 21, 'Everton': 11, 'West Ham': 6, 'West Brom': 5, 'Fulham': 12, 'Aston Villa': 10, 'Watford': 24, 'Crystal Palace': 8, 'Liverpool': 1, 'Man United': 19, 'Hull': 13, 'Norwich': 2, 'Sunderland': 3, 'Chelsea': 7, 'Leicester': 20, 'Tottenham': 18, 'Stoke': 16, 'Burnley': 22, 'Swansea': 4, 'Cardiff': 17, 'Arsenal': 0, 'Bournemouth': 23, 'Southampton': 15, 'Newcastle': 14, 'Man City': 9}


In [242]:
X,y = c.getH4(5)


h4:start format
 progress 5033


TypeError: 'NoneType' object is not iterable

In [218]:
%matplotlib qt
def plotErrorDate(X_test, X_err, dateCol = 10):
    X_test_date =np.sort(X_test[:,c.dateColumn])
    X_date=[]
    y_date=[]
    for v in X_test_date:
        date = datetime.datetime.fromtimestamp(v*24*60*60)
        if len(X_date) ==0  or X_date[-1] != date:
            X_date.append(date)
            y_date.append(1)
        else:
            y_date[-1] = y_date[-1] +1
    plt.plot_date(X_date,y_date,xdate=True)
    X_err_d = np.sort(X_err[:,c.dateColumn])
    X_err_date=[]
    y_err_date = []
    for v in X_err_d:
        date = datetime.datetime.fromtimestamp(v*24*60*60)
        if len(X_err_date) ==0  or X_err_date[-1] != date:
            X_err_date.append(date)
            y_err_date.append(1)
        else:
            y_err_date[-1] = y_err_date[-1] +1
    plt.plot_date(X_err_date,y_err_date,xdate=True,color='red')
    plt.show()



In [ ]:
inData = c.readPredict("predict.csv")
print(inData)
X_in, y_in = c.getH4(4,target=inData)
print(y_in)


In [230]:
from sklearn.neural_network import MLPClassifier
from sklearn.learning_curve import learning_curve
from custom import SoftMaxMLPClassifier
from sklearn.pipeline import Pipeline
g_hiddenNodes = int(60)
g_alpha = 35
clf = SoftMaxMLPClassifier(hidden_layer_sizes=[g_hiddenNodes], activation='logistic', algorithm='l-bfgs', alpha=g_alpha, 
              learning_rate_init=0.01,learning_rate='adaptive' ,max_iter=1000,early_stopping = True,verbose = 3)
mlp = Pipeline([('scl', StandardScaler()),('clf', clf)])

In [220]:
mlp.fit(X,y)
#res = mlp.predict(X_in)

Pipeline(steps=[('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('clf', SoftMaxMLPClassifier(activation='logistic', algorithm='l-bfgs', alpha=35,
           batch_size='auto', beta_1=0.9, beta_2=0.999,
           early_stopping=True, epsilon=1e-08, hidden_layer_sizes=[60],
           learn...e=None, shuffle=True, tol=0.0001,
           validation_fraction=0.1, verbose=3, warm_start=False))])

In [ ]:
print(X.shape)
print(X_in.shape)
res = mlp.predict(X_in)
print(res)
proba= mlp.predict_proba(X_in)
print(y_in)
print(mlp.score(X_in,y_in))
print(proba)
print(firstNScore(2,proba,y_in))

In [ ]:
#future
inData = c.readPredict("future.csv")
print(inData)
X_in, y_in = c.getH4(4,target=inData)
res = mlp.predict(X_in)
proba= mlp.predict_proba(X_in)
print ("{}\n{}".format(res,proba))

In [ ]:
print ("start learning")
sys.stdout.flush()
train_sizes, train_scores, test_scores = learning_curve(estimator=mlp, 
                       X=X, 
                      y=y, 
                      train_sizes=np.linspace(0.1, 1.0, 4), 
                      cv=4,
                     n_jobs=1,verbose=3)
print("finishing")   
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
plotCurve(train_mean,train_std,test_mean,test_std,train_sizes)

In [ ]:
from sklearn.cross_validation import StratifiedKFold
def crossValidate(mlp, X,y, fold = 10):
    y_label = np.argmax(y,axis=1)

    kfold = StratifiedKFold(y=y_label, 
                             n_folds=fold,
                            random_state=1)

    scores = []
    train_scores=[]
    firstNScores = []
    for k, (train, test) in enumerate(kfold):

        mlp.fit(X[train], y[train])
        score = mlp.score(X[test], y[test])
        firstNScores.append(firstNScore(2, mlp.predict_proba(X[test]), y[test]))
        train_scores.append(mlp.score(X[train],y[train]))
        scores.append(score)
        print('Fold: %s, Class dist.: %s, Acc: %.3f' % (k+1, 
                    np.bincount(y_label[train]), score))    
        
        
    return train_scores,scores, firstNScores



In [ ]:

def lamda_test(mlp, X, y, lamdas):
    
    train_scores=[]
    test_scores=[]
    for lamda in lamdas:
        clf.set_params(alpha= lamda)
       # train_s, test_s, firstNScores = crossValidate(mlp,X,y,fold=5)
        train_s, test_s, firstNScores =futureTest(mlp,X,y,numOfWeek=15) 
        train_scores.append(train_s)
        test_scores.append(test_s)
        print("lamda: {}, train: {}, test: {}".format(lamda, 
                    np.mean(train_s), np.mean(test_s)) )
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plotCurve(train_mean,train_std,test_mean,test_std,lamdas)
    return np.array(train_scores),np.array(test_scores)

l_range = np.array(range(10,70,10))
train_scores,test_scores = lamda_test(mlp,X,y,l_range)
#50

In [ ]:
import matplotlib.pyplot as plt
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)


#plotCurve(train_mean,train_std,test_mean,test_std,l_range)
plt.plot(l_range, train_mean, 
        color='blue', marker='o', 
        markersize=5, 
        label='training accuracy')
plt.fill_between(l_range, 
              train_mean + train_std,
               train_mean - train_std, alpha=0.15, color='blue')

plt.plot(l_range, test_mean, 
          color='green', linestyle='--', 
          marker='s', markersize=5, 
         label='validation accuracy')
plt.fill_between(l_range, 
                  test_mean + test_std,
                 test_mean - test_std, 
                alpha=0.15, color='green')
plt.xlabel('lamda')
plt.ylabel('Accuracy')
plt.show()


In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_test , y_train, y_test = train_test_split(X,y,test_size =0.3)



In [ ]:
clf.set_params(alpha=50)
mlp.fit(X_train, y_train)

In [ ]:
res = mlp.predict_proba(X_test)


In [ ]:
fig = plt.gcf()
fig.set_size_inches(18.5, 10.5)
predicted = mlp.predict(X_test)
print(mlp.score(X_test, y_test))
proba = mlp.predict_proba(X_test)
print(firstNScore(2,proba,y_test))
error = X_test[np.argmax(predicted,axis=1) != np.argmax(y_test,axis=1)]
plotErrorDate(oneHotDecode(c,X_train),oneHotDecode(c,error))

In [ ]:
likehoodScore(res,y_test)

In [ ]:
home_teams_test = np.argmax(X_test[:,0:len(c.teamsMap)],axis=1)
away_teams_test = np.argmax(X_test[:,len(c.teamsMap):len(c.teamsMap)*2],axis=1)
inverseMap ={}
for name in c.teamsMap.keys():        
    inverseMap[c.teamsMap[name]] = name

home_name =[]
away_name =[]
for i in range(home_teams_test.shape[0]):         
    home_name.append(inverseMap[home_teams_test[i]])
    away_name.append(inverseMap[away_teams_test[i]])
pre_y = np.argmax(predicted,axis=1)
real_y = np.argmax(y_test,axis=1)
for i in range(predicted.shape[0]):
    if pre_y[i] != real_y[i]:
        print('-------------')
        print("{} vs {}".format(home_name[i],away_name[i]))
        print("{} vs {}".format(res[i],y_test[i]))
    

In [ ]:
#c.df[(c.df['HomeTeam'] == 'West Brom') | (c.df['AwayTeam']=='West Brom')]

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.learning_curve import learning_curve
from custom import SoftMaxMLPClassifier
from sklearn.pipeline import Pipeline
%matplotlib inline
import matplotlib.pyplot as plt
def testNodeSize(start ,end):
    node_range = range(start,end,25)
    train_means = []
    train_std = []
    test_means =[]
    test_std=[]
    for node in node_range:   
        print("start node:{}".format(node))
        clf = SoftMaxMLPClassifier(hidden_layer_sizes=[node], activation='logistic', algorithm='l-bfgs', alpha=30, 
                  learning_rate_init=0.01,learning_rate='adaptive' ,max_iter=500,early_stopping = True,verbose = 3)
        mlp = Pipeline([('scl', StandardScaler()),('clf', clf)])
       # train_scores,test_scores = crossValidate(mlp,X,y,fold=5)
        train_scores,test_scores , first2= futureTest(mlp, X,y,numOfWeek = 10)
        train_means.append(np.mean(train_scores))
        train_std.append(np.std(train_scores))
        test_means.append(np.mean(test_scores))
        test_std.append(np.std(test_scores))
        print("Node {}: train_mean {}  v.s. test_mean {}".format(node,np.mean(train_scores),np.mean(test_scores)))
    plotCurve(np.array(train_means),np.array(train_std),np.array(test_means),np.array(test_std),np.array(node_range))
    return train_means,train_std,test_means,test_std

In [ ]:
train_means,train_std,test_means,test_std=testNodeSize(10 ,X.shape[1] )

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.learning_curve import learning_curve
from custom import SoftMaxMLPClassifier
from sklearn.pipeline import Pipeline
%matplotlib inline
import matplotlib.pyplot as plt
def testRecentNum(start, end):
    recent_range = range(start,end)
    train_means = []
    train_std = []
    test_means =[]
    test_std=[]
    first2_mean=[]
    for recent in recent_range:
        print("start recent:{}".format(recent))
        X,y = c.getH4(recent)
        clf = SoftMaxMLPClassifier(hidden_layer_sizes=[60], activation='logistic', algorithm='l-bfgs', alpha=30, 
                  learning_rate_init=0.01,learning_rate='adaptive' ,max_iter=500,early_stopping = True,verbose = 3)
        mlp = Pipeline([('scl', StandardScaler()),('clf', clf)])
        #train_scores,test_scores, first2 = crossValidate(mlp,X,y,fold=10)
        train_scores,test_scores, first2 = futureTest(mlp, X,y,numOfWeek = 15)
        train_means.append(np.mean(train_scores))
        train_std.append(np.std(train_scores))
        test_means.append(np.mean(test_scores))
        test_std.append(np.std(test_scores))
        first2_mean.append(np.mean(first2))
        print("recent {}: train_mean {}  v.s. test_mean {} , first2_mean {}".format(
                recent,np.mean(train_scores),np.mean(test_scores),np.mean(first2)))
    plotCurve(np.array(train_means),np.array(train_std),np.array(test_means),np.array(test_std),np.array(recent_range))
    return train_means,train_std,test_means,test_std,first2_mean



In [ ]:
train_means,train_std,test_means,test_std,first2_mean=testRecentNum(0 ,10)

In [234]:

clf.set_params(alpha=g_alpha)
futureTest(mlp,X,y,numOfWeek = 20,verbose=True)      


week0
numOftest 5 , score 0.6
[['Norwich' 'Man City' '0.06849064915829184' '0.09544549248184837'
  '0.8360638583598599' 'False' 'True' 'False']
 ['Aston Villa' 'Tottenham' '0.12696628052319248' '0.5362798115073082'
  '0.33675390796949944' 'False' 'False' 'True']]
first2 : {} 1.0
week1
numOftest 10 , score 0.3
[['Chelsea' 'Stoke' '0.9495952848450628' '0.024951646195902004'
  '0.025453068959034996' 'False' 'True' 'False']
 ['Everton' 'West Ham' '0.5027482050602833' '0.32493776656651074'
  '0.17231402837320603' 'False' 'False' 'True']
 ['Newcastle' 'Bournemouth' '0.1279799839986163' '0.5146840766203895'
  '0.3573359393809941' 'False' 'False' 'True']
 ['Southampton' 'Sunderland' '0.8137664508876106' '0.05832414814042185'
  '0.12790940097196754' 'False' 'True' 'False']
 ['Tottenham' 'Arsenal' '0.4063555326676663' '0.3886135435511619'
  '0.2050309237811719' 'False' 'True' 'False']
 ['Watford' 'Leicester' '0.5966845935761861' '0.12369219980213225'
  '0.2796232066216816' 'False' 'False' 'True'

(0.71217371217371217, 0.46478873239436619, 0)

In [ ]:

proba = mlp.predict_proba(X_test)
precisionMatrix(proba,y_test)



In [233]:
#future
mlp.fit(X,y)
inData = c.readPredict("future.csv")
print(inData)
X_in, y_in = c.getH4(5,target=inData)
res = mlp.predict(X_in)
proba= mlp.predict_proba(X_in)
print(mlp.score(X_in,y_in))
print (np.hstack([proba,y_in]))


  Div       Date        HomeTeam     AwayTeam  FTHG  FTAG  FTR  HTHG  HTAG  \
0  E0 2016-03-19         Chelsea     West Ham     2     2    1     1     1   
1  E0 2016-03-19  Crystal Palace    Leicester     0     1    2     0     1   
2  E0 2016-03-19         Everton      Arsenal     0     2    2     0     2   
3  E0 2016-03-19         Swansea  Aston Villa     1     0    0     0     0   
4  E0 2016-03-19         Watford        Stoke     1     2    2     0     1   
5  E0 2016-03-19       West Brom      Norwich     0     1    2     0     0   
6  E0 2016-03-20        Man City   Man United     0     1    2     0     1   
7  E0 2016-03-20       Newcastle   Sunderland     1     1    1     0     1   
8  E0 2016-03-20     Southampton    Liverpool     3     2    0     0     2   
9  E0 2016-03-20       Tottenham  Bournemouth     3     0    0     2     0   

  HTR        Referee  
0   D       R Madley  
1   A        M Jones  
2   A  M Clattenburg  
3   D         M Dean  
4   A       C Pawson  
5  

In [235]:
print(c.dateColumn)

12


In [236]:
print(X.shape)

(978, 321)
